In [1]:
import google.auth

credentials, project_id = google.auth.default()

# install dependencies
# pip install cloud-sql-python-connector["pymysql"] SQLAlchemy==2.0.7

import sys
if '..' not in sys.path:
    sys.path.append('..')

from scripts.utils import openfile
from scripts.utils import Hosts, Plays, Programs, Shows, Timeslots


from google.cloud.sql.connector import Connector

import sqlalchemy
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.expression import func

from creds import user

import requests
import pandas as pd

In [2]:
# initialize parameters
INSTANCE_CONNECTION_NAME = user['INSTANCE_CONNECTION_NAME']# i.e demo-project:us-central1:demo-instance
print(f"Your instance connection name is: {INSTANCE_CONNECTION_NAME}")
DB_USER = user['DB_USER']
DB_PASS = user['DB_PASS']
DB_NAME = user['DB_NAME']


root_path = user['root_path']

Your instance connection name is: kexp-explore:us-central1:kexp-sandbox


In [3]:
# initialize Connector object
connector = Connector()

# function to return the database connection object
def getconn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pymysql",
        user=DB_USER,
        password=DB_PASS,
        db=DB_NAME
    )
    return conn

# create connection pool with 'creator' argument to our connection object function
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)

engine = pool

In [4]:
r = requests.get('https://api.kexp.org/v2/')
r.status_code

urls = r.json()
urls

{'hosts': 'https://api.kexp.org/v2/hosts/',
 'programs': 'https://api.kexp.org/v2/programs/',
 'shows': 'https://api.kexp.org/v2/shows/',
 'plays': 'https://api.kexp.org/v2/plays/',
 'timeslots': 'https://api.kexp.org/v2/timeslots/'}

### Programs Populate

In [5]:
programs = requests.get(urls['programs'])
programs_json=programs.json()
print(f"Total Entries: {programs_json['count']}")

Total Entries: 40


In [6]:
programs_json

{'count': 40,
 'next': 'https://api.kexp.org/v2/programs/?limit=20&offset=20',
 'previous': None,
 'results': [{'id': 26,
   'uri': 'https://api.kexp.org/v2/programs/26/',
   'name': '90.TEEN',
   'description': 'Rock,Eclectic,Variety Mix',
   'tags': 'Rock,Eclectic,Variety Mix',
   'is_active': True},
  {'id': 38,
   'uri': 'https://api.kexp.org/v2/programs/38/',
   'name': 'Astral Plane',
   'description': '',
   'tags': 'Rock,Eclectic,Psychedelic',
   'is_active': True},
  {'id': 1,
   'uri': 'https://api.kexp.org/v2/programs/1/',
   'name': 'Audioasis',
   'description': "Audioasis is KEXP's long-running Northwest music show hosted by Sharlese Metcalf. Past hosts have included Jonathen Poneman, Jason Hughes, Scott Vanderpool, Stevie Zoom, Sean Nelson and Hannah Levin. John Richards helps produce the show. Audioasis delivers three hours of local and live bands from all areas and genres of the Northwest. On Audioasis, you'll hear the new, the old, the demos, the vinyl, and the future

In [7]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
    
# Insert data into the table
try:
    for data in programs_json['results']:
        program = Programs(**data)
        session.add(program)

    # Commit the transaction
    session.commit()
    print("Data inserted successfully!")

except Exception as e:
    # Roll back the transaction if an error occurs
    session.rollback()
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

Error: (pymysql.err.IntegrityError) (1062, "Duplicate entry '26' for key 'programs.PRIMARY'")
[SQL: INSERT INTO programs (id, uri, name, description, tags, is_active) VALUES (%(id)s, %(uri)s, %(name)s, %(description)s, %(tags)s, %(is_active)s)]
[parameters: [{'id': 26, 'uri': 'https://api.kexp.org/v2/programs/26/', 'name': '90.TEEN', 'description': 'Rock,Eclectic,Variety Mix', 'tags': 'Rock,Eclectic,Variety Mix', 'is_active': 1}, {'id': 38, 'uri': 'https://api.kexp.org/v2/programs/38/', 'name': 'Astral Plane', 'description': '', 'tags': 'Rock,Eclectic,Psychedelic', 'is_active': 1}, {'id': 1, 'uri': 'https://api.kexp.org/v2/programs/1/', 'name': 'Audioasis', 'description': "Audioasis is KEXP's long-running Northwest music show hosted by Sharlese Metcalf. Past hosts have included Jonathen Poneman, Jason Hughes, Scott Vand ... (239 characters truncated) ... os, the vinyl, and the future of music. Every first Saturday of the month you can catch Audioasis out in the city LIVE! Tune in Satur

In [8]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
# Query the table for 5 random rows
try:
    random_rows = session.query(Programs).order_by(func.rand()).limit(5).all()
    for row in random_rows:
        print(row.id, row.uri, row.name, row.description, row.tags, row.is_active)

except Exception as e:
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

38 https://api.kexp.org/v2/programs/38/ Astral Plane  Rock,Eclectic,Psychedelic True
8 https://api.kexp.org/v2/programs/8/ Preachin' the Blues From out of the swamp and through the woods comes the Preachin' the Blues show. Every Sunday I explore the downhome, uptown and gospel sounds of rhythm and blues music, and the focus is always on the most soulful, gritty, heartfelt and creative artists and material. \n\nCountry blues gets a special spotlight in the early morning, but quite often we move into the church for some old time singing, powerful preaching and funky accompaniment. \n\nI am a fan of southern soul and I always include something from my collection of Stax and Hi stuff in the mix, but the deep blues are firmly represented in each show with heaping helpings of the three Kings of the blues guitar, the rich character of Chicago's legends, and always a certain amount of Louisiana's magickal mystique. \n\n-Johnny Horn Blues,Country,Roots False
22 https://api.kexp.org/v2/programs/

In [9]:
while programs_json['next'] != None:
    programs = requests.get(programs_json['next'])
    programs_json=programs.json()
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
        
    # Insert data into the table
    try:
        for data in programs_json['results']:
            user = Programs(**data)
            session.add(user)

        # Commit the transaction
        session.commit()
        print("Data inserted successfully!")

    except Exception as e:
        # Roll back the transaction if an error occurs
        session.rollback()
        print(f"Error: {e}")

    finally:
        # Close the session
        session.close()

Data inserted successfully!


### Shows Populate

In [10]:
shows = requests.get(urls['shows'])
shows_json=shows.json()
print(f"Total Entries: {shows_json['count']}")

Total Entries: 59731


In [12]:
shows_json['results'][0]

{'id': 59753,
 'uri': 'https://api.kexp.org/v2/shows/59753/',
 'program': 18,
 'program_uri': 'https://api.kexp.org/v2/programs/18/',
 'hosts': [43],
 'host_uris': ['https://api.kexp.org/v2/hosts/43/'],
 'program_name': 'Variety Mix',
 'program_tags': 'Rock,Eclectic,Variety Mix',
 'host_names': ['Troy Nelson'],
 'tagline': '',
 'image_uri': 'https://www.kexp.org/filer/canonical/1583367959/22030/',
 'start_time': '2024-03-23T15:01:26-07:00'}

In [15]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
    
# Insert data into the table
try:
    for data in shows_json['results']:
        # Keys to drop
        keys_to_drop = ['host_uris', 'host_names']
        # New dictionary without the specified keys
        data = {key: value for key, value in data.items() if key not in keys_to_drop}
        show = Shows(**data)
        session.add(show)

    # Commit the transaction
    session.commit()
    print("Data inserted successfully!")

except Exception as e:
    # Roll back the transaction if an error occurs
    session.rollback()
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

Data inserted successfully!


In [16]:
for _ in range(5):
    shows = requests.get(shows_json['next'])
    shows_json=shows.json()
    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()
        
    # Insert data into the table
    try:
        for data in shows_json['results']:
            # Keys to drop
            keys_to_drop = ['host_uris', 'host_names']
            # New dictionary without the specified keys
            data = {key: value for key, value in data.items() if key not in keys_to_drop}
            show = Shows(**data)
            session.add(show)

        # Commit the transaction
        session.commit()
        print("Data inserted successfully!")

    except Exception as e:
        # Roll back the transaction if an error occurs
        session.rollback()
        print(f"Error: {e}")

    finally:
        # Close the session
        session.close()

Data inserted successfully!
Data inserted successfully!
Data inserted successfully!
Data inserted successfully!
Data inserted successfully!


### Plays Populate

In [5]:
shows = requests.get(urls['plays'])
plays_json=shows.json()


In [6]:
plays_json['results'][0]

{'id': 3326526,
 'uri': 'https://api.kexp.org/v2/plays/3326526/',
 'airdate': '2024-03-25T07:38:30-07:00',
 'show': 59770,
 'show_uri': 'https://api.kexp.org/v2/shows/59770/',
 'image_uri': 'https://ia804704.us.archive.org/9/items/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee-2200354304_thumb500.jpg',
 'thumbnail_uri': 'https://ia904704.us.archive.org/9/items/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee-2200354304_thumb250.jpg',
 'song': 'Deep Red Bells',
 'track_id': '3087c18d-c59f-37e4-87bc-17b6683c509c',
 'recording_id': '74a939b9-995b-4bc6-8393-1c52aae295d9',
 'artist': 'Neko Case',
 'artist_ids': ['e13d2935-8c42-4c0a-96d7-654062acf106'],
 'album': 'Blacklisted',
 'release_id': '0c5fc37e-c75c-49ee-9609-e69e7c0743ee',
 'release_group_id': 'ec08962a-433c-3071-9978-488e44a25a3b',
 'labels': ['Bloodshot Records'],
 'label_ids': ['c1a4129a-95dd-4e8c-b42e-7cd9fc3c208b'],
 'release_date': '2002-08-20',
 'rotation

In [11]:
# Create a session
Session = sessionmaker(bind=engine)
session = Session()
    
# Insert data into the table
try:
    for data in plays_json['results']:
        if 'song' in data.keys():
            # Rename Keys
            data['show_id'] = data['show']
            # Keys to drop
            keys_to_drop = ['show', 'show_uri', 'labels', 'label_ids', 'rotation_status', 'play_type','image_uri', 'thumbnail_uri', 'artist_ids']

            # New dictionary without the specified keys
            data = {key: value for key, value in data.items() if key not in keys_to_drop}
            play = Plays(**data)
            session.add(play)

    # Commit the transaction
    session.commit()
    print("Data inserted successfully!")

except Exception as e:
    # Roll back the transaction if an error occurs
    session.rollback()
    print(f"Error: {e}")

finally:
    # Close the session
    session.close()

Error: (pymysql.err.IntegrityError) (1048, "Column 'track_id' cannot be null")
[SQL: INSERT INTO plays (id, uri, airdate, show_id, song, track_id, recording_id, artist, album, release_id, release_group_id, release_date, is_local, is_request, is_live, comment) VALUES (%(id)s, %(uri)s, %(airdate)s, %(show_id)s, %(song)s, %(track_id)s, %(recording_id)s, %(artist)s, %(album)s, %(release_id)s, %(release_group_id)s, %(release_date)s, %(is_local)s, %(is_request)s, %(is_live)s, %(comment)s)]
[parameters: [{'id': 3326526, 'uri': 'https://api.kexp.org/v2/plays/3326526/', 'airdate': '2024-03-25T07:38:30-07:00', 'show_id': 59770, 'song': 'Deep Red Bells', 'track_id': '3087c18d-c59f-37e4-87bc-17b6683c509c', 'recording_id': '74a939b9-995b-4bc6-8393-1c52aae295d9', 'artist': 'Neko Case', 'album': 'Blacklisted', 'release_id': '0c5fc37e-c75c-49ee-9609-e69e7c0743ee', 'release_group_id': 'ec08962a-433c-3071-9978-488e44a25a3b', 'release_date': '2002-08-20', 'is_local': 1, 'is_request': 0, 'is_live': 0, 'co

In [9]:
for data in plays_json['results']:
    if 'song' in data.keys() and :
        print(data)

{'id': 3326525, 'uri': 'https://api.kexp.org/v2/plays/3326525/', 'airdate': '2024-03-25T07:38:11-07:00', 'show': 59770, 'show_uri': 'https://api.kexp.org/v2/shows/59770/', 'image_uri': '', 'thumbnail_uri': '', 'comment': '', 'play_type': 'airbreak', 'show_id': 59770}
{'id': 3326520, 'uri': 'https://api.kexp.org/v2/plays/3326520/', 'airdate': '2024-03-25T07:25:33-07:00', 'show': 59770, 'show_uri': 'https://api.kexp.org/v2/shows/59770/', 'image_uri': '', 'thumbnail_uri': '', 'comment': '', 'play_type': 'airbreak', 'show_id': 59770}
{'id': 3326515, 'uri': 'https://api.kexp.org/v2/plays/3326515/', 'airdate': '2024-03-25T07:01:20-07:00', 'show': 59770, 'show_uri': 'https://api.kexp.org/v2/shows/59770/', 'image_uri': '', 'thumbnail_uri': '', 'comment': '', 'play_type': 'airbreak', 'show_id': 59770}
{'id': 3326512, 'uri': 'https://api.kexp.org/v2/plays/3326512/', 'airdate': '2024-03-25T06:51:13-07:00', 'show': 59769, 'show_uri': 'https://api.kexp.org/v2/shows/59769/', 'image_uri': '', 'thumbn

In [10]:
plays_json['results']


[{'id': 3326526,
  'uri': 'https://api.kexp.org/v2/plays/3326526/',
  'airdate': '2024-03-25T07:38:30-07:00',
  'show': 59770,
  'show_uri': 'https://api.kexp.org/v2/shows/59770/',
  'image_uri': 'https://ia804704.us.archive.org/9/items/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee-2200354304_thumb500.jpg',
  'thumbnail_uri': 'https://ia904704.us.archive.org/9/items/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee/mbid-0c5fc37e-c75c-49ee-9609-e69e7c0743ee-2200354304_thumb250.jpg',
  'song': 'Deep Red Bells',
  'track_id': '3087c18d-c59f-37e4-87bc-17b6683c509c',
  'recording_id': '74a939b9-995b-4bc6-8393-1c52aae295d9',
  'artist': 'Neko Case',
  'artist_ids': ['e13d2935-8c42-4c0a-96d7-654062acf106'],
  'album': 'Blacklisted',
  'release_id': '0c5fc37e-c75c-49ee-9609-e69e7c0743ee',
  'release_group_id': 'ec08962a-433c-3071-9978-488e44a25a3b',
  'labels': ['Bloodshot Records'],
  'label_ids': ['c1a4129a-95dd-4e8c-b42e-7cd9fc3c208b'],
  'release_date': '2002-